In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [4]:
#FOURSQUARE IDs 
CLIENT_ID = '3S1PFTNL1HAFWEYMYHRLCVZVWN1DSK0IPHKSX5JXAGR3E1ML' # your Foursquare ID
CLIENT_SECRET = 'V4IEPFK4NJZZER4LJA33R2LX3S5KRBHVECAFZJDVTLFZUTBP' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

## Capstone Final Project 

### Subject: Social places vs Covid19 
Question I want to address is : Can we see relationship between large cities' social places (such as  bar, restaurant, cinema, theatre, ... ) and postive Covid19 cases in France ?
How to: I'm going to get  the list of biggest cities on France, get the numbers about the Covid19 cases in France, get the top most visited social places information from FourSquare for these cities. Aggregating these different data sets, I will leverage the clustering techniques to highlight similarities and hopefully get some insight of my original question. 
My assumption is that, the more we would have Social places the highest would be the number of Covid positive cases.. 


### Business problem
It is obvious I believe the economical impact of closing everything down. There are many different shopts being closed eventhough the medical protective measure were in place. Economical impact oculd be even bigger than the medical impact. Looking at the problem by mixing the most visited places and types of places vs the number of cases can shade some lights on what are the places we could open again.

### Data: 
I will gather  Covid related data: numbers of positive cases by region/department, number of Covid test  per region/departement to reduce the bias of number of positive vs number of tested.
<br>I will list of cities from which I will take the largest ( most representative ) for each department
<br>I will use the Foursquare API to get the "Social" places around the biggest cities 


#### Details on each data set to be used for this study:

#### Covid stats : pulling data from the governement portal

From May to December 2020
https://www.data.gouv.fr/fr/datasets/donnees-relatives-aux-resultats-des-tests-virologiques-covid-19/
file: https://static.data.gouv.fr/resources/donnees-relatives-aux-resultats-des-tests-virologiques-covid-19/20201211-192020/sp-pos-quot-dep-2020-12-11-19h20.csv
url pointing to daily updates ( permanent url for latest info ): https://www.data.gouv.fr/fr/datasets/r/406c6a23-e283-4300-9484-54e78c8ae675
where:
- dep	is the departement ,
- jour is the date of data collection 	
- P	is positive tested results of the given date
- T	is tested in total of the given date
- cl_age90	: this attributes is not clearly explain I will then ignore it.
- pop is the total  population in that departement 


#### French Cities  list per departments: : pulling data from the governement portal

https://www.data.gouv.fr/fr/datasets/regions-departements-villes-et-villages-de-france-et-doutre-mer/
==> direct link to the data csv data set:https://www.data.gouv.fr/fr/datasets/r/a8adb872-dfac-46b1-9885-da8ca7d2d931, from where I will use the cities.csv 
where the below attributes are:
- id : unique ID
- department_cod : numeric code of the department where the city is
- insee_code : code being used for statistic city identifier
- zip_code : Postal code of the city
- name : Name of the department 
- slug : Name of the department without special character inside
- gps_lat	gps_lng  : coordinates of the City

largest cities (  based on population ):
https://fr.wikipedia.org/wiki/Liste_des_communes_de_France_les_plus_peupl%C3%A9es
I'll take only the Year 2017 as it is the most recent 
From that url, I will use the table  named 'Communes de plus de 30 000 habitants' , and I will used the INSEE code to merge de data set with the previous data set so I can restrict my study to largest cities only.


#### Crisis Contextual information , to bring more insights 
Differents action in lockdown situation in France. I may input these information to refine observation and see if there are drops in the number of positive Covid tests
https://www.lemonde.fr/les-decodeurs/article/2020/05/12/coronavirus-de-la-chauve-souris-au-deconfinement-la-chronologie-de-la-pandemie_6039448_4355770.html
https://fr.wikipedia.org/wiki/Chronologie_de_la_pand%C3%A9mie_de_Covid-19_en_France
I will use these information on a visual representation to see the different influence ( or not ) of the lockdown situation on the positive tests, assuming I have enough data to cross at the same period.



## Read the Covid data

In [5]:
import io
import requests

covid_url = "https://www.data.gouv.fr/fr/datasets/r/406c6a23-e283-4300-9484-54e78c8ae675"

src = requests.get(covid_url).content  # get raw formatted content data from that url
covid_data = pd.read_csv(io.StringIO(src.decode('utf-8')),sep=';',low_memory=False) # pass to pandas read_csv a 'file-like' object the raw we got from prev step

## Convert the jour to a known date formate for later use
covid_data.jour = pd.to_datetime(covid_data.jour)

##Calculate here the ratio Postive vs Tested to have a relative value to compare afterwards
covid_data['PtoT_ratio'] = covid_data['P']/covid_data['T']
covid_data['PtoT_ratio'].fillna(0,inplace=True) # put 0 whenever there is no data available in P or T
## Calculate positive vs the population of a given Department 
covid_data['PtoPop_ratio'] = covid_data['P']/covid_data['pop']

# This data set contains all the data related to Covid tests since May 2020
display (covid_data.tail())

# How many departement do we have:
print (" We have ",len(covid_data.dep.unique())," département in Covid dataset")
departement_covid = covid_data.dep.unique()


### Refine this dataset to keep only the average of PtoPop_ratio, as this is the data we want to  include in our study . Meaning what is the average Positive cases detection in regards to the Social places
covid_data_refine = covid_data.groupby(['dep']).agg({'PtoT_ratio':'mean'}).reset_index()

display (covid_data_refine.tail())

,dep,jour,P,T,cl_age90,pop,PtoT_ratio,PtoPop_ratio
266547,978,2020-12-31,0,12,69,3060.024992,0.000000,0.000000
266548,978,2020-12-31,0,7,79,1152.934377,0.000000,0.000000
266549,978,2020-12-31,0,0,89,346.012162,0.000000,0.000000
266550,978,2020-12-31,0,0,90,70.655167,0.000000,0.000000
266551,978,2020-12-31,3,143,0,35334.000000,0.020979,0.000085


 We have  104  département in Covid dataset


,dep,PtoT_ratio
99,974,0.019651
100,975,0.003647
101,976,0.085109
102,977,0.011322
103,978,0.053099


In [6]:
## Last date entry from my Covid data source 
display ('Latest data entry date we have in the covid data ? ')
display (covid_data['jour'].unique()[-1])


'Latest data entry date we have in the covid data ? '

numpy.datetime64('2020-12-31T00:00:00.000000000')

## Get the coordinate of cities in France
### this dataset have been taken from the zip availabile in https://www.data.gouv.fr/fr/datasets/r/a8adb872-dfac-46b1-9885-da8ca7d2d931 , zip file which contains other file so I extracted that specific one

In [7]:
import io
import requests

localfile = "C:/Users/JULIENSauvanet/Documents/JSAUVANET/Coursera/Projects/Coursera_Capstone/cities.csv"
french_cities_coordinates = pd.read_csv(localfile,sep=',',low_memory=False) # pass to pandas read_csv a 'file-like' object the raw we got from prev step . 

## Format the zip code to int
## Before I need to take care of those without Zip code from the TOM/DOM
french_cities_coordinates.zip_code.fillna('00000',inplace=True)
french_cities_coordinates.zip_code.astype(np.int64)

french_cities_coordinates.dtypes
display (french_cities_coordinates.head())
# How many departement do we have:
print (" We have ",len(french_cities_coordinates.department_code.unique())," département in City Coordinates dataset")

,id,department_code,insee_code,zip_code,name,slug,gps_lat,gps_lng
0,1,01,01001,1400,L'Abergement-Clémenciat,l abergement clemenciat,46.156782,4.924699
1,2,01,01002,1640,L'Abergement-de-Varey,l abergement de varey,46.010086,5.428759
2,3,01,01004,1500,Ambérieu-en-Bugey,amberieu en bugey,45.958409,5.375992
3,4,01,01005,1330,Ambérieux-en-Dombes,amberieux en dombes,46.000120,4.910602
4,5,01,01006,1300,Ambléon,ambleon,45.746427,5.602492


 We have  109  département in City Coordinates dataset


## Get the dataset to providing the largest cities in France

In [8]:
url = "https://fr.wikipedia.org/wiki/Liste_des_communes_de_France_les_plus_peupl%C3%A9es"
tables = pd.read_html(url, encoding='utf-8') # Returns list of all tables on page
# First table of that page is what we need .. 
big_french_cities = tables[0] 

# Cleaning the old Population statscolumns as I need only the most recent year ( 7th columns)
nbofcols= big_french_cities.shape[1]
col = 0
while col < 7:
    nbofcols = nbofcols - 1
    big_french_cities.drop(big_french_cities.columns[nbofcols], axis='columns',inplace=True)
    big_french_cities.shape[1]
    col = col +1

## Cleaning the Multiu Columns level
big_french_cities.columns = big_french_cities.columns.droplevel(1)

## Cleaning the Population numbers format to get rid of the x08F ==> str.replace('[^\x00-\x8F]', '')
big_french_cities['Population légale'] = big_french_cities['Population légale'].str.replace('[^\x00-\x8F]', '')
## get rid of non standard format in Population légale , found sometime there are info into brakets.. need to clean this up
big_french_cities['Population légale'] = big_french_cities['Population légale'].str.replace(r"\(.*\)","")
big_french_cities['Population légale'] = big_french_cities['Population légale'].astype(np.int64)

#print (big_french_cities.dtypes)
display (big_french_cities.tail())

# Do we have one "big" city for each department ?
print (" We have ",len(big_french_cities['Département'].unique())," département in Big cities dataset")

departement_big_french_cities = big_french_cities['Département'].unique()

,Rang2021,CodeInsee,Commune,Département,Statut,Région,Population légale
276,273,24322,Périgueux,Dordogne,Préfecture,Nouvelle-Aquitaine,30060
277,274,94018,Charenton-le-Pont,Val-de-Marne,--,Île-de-France,30053
278,275,92064,Saint-Cloud,Hauts-de-Seine,--,Île-de-France,30038
279,276,54547,Vandœuvre-lès-Nancy,Meurthe-et-Moselle,--,Grand Est,30009
280,277,93077,Villemomble,Seine-Saint-Denis,--,Île-de-France,30005


 We have  88  département in Big cities dataset


In [11]:
print ("Merge two data sets: largest cities and list of cities with their coordinates ( based on Insee Code)" )
## big_french_cities => CodeInsee & french_cities_coordinates => insee_code
# rename header for the merge operation
big_french_cities.rename(index=str, columns={"CodeInsee": "insee_code"}, inplace = True)

# merge our datafile wth the coordinate dataframe
fr_cities = pd.merge(big_french_cities, french_cities_coordinates, on='insee_code', how='inner') # Join info on PostalCode which is our unique value, common to both dataframe

# remove duplicates if any - I've found that multiple lines appears for same Commune with with different gps coordinates ( having multiple zip code within same City). which is useless in our case
fr_cities.drop_duplicates(subset=['insee_code','Commune','Département','department_code'],inplace=True)

#########################################
#########################################
#########################################
#########################################
#########################################
#Restrict the list for testing purposes
#tmp = fr_cities.loc[(fr_cities.Commune.isin(['Nice','Paris','Marseille','Lille']))] ##Limiting to Nice for now to preserve FourSquare calls while testing
#fr_cities = tmp
#########################################
#########################################
#########################################
#########################################

# drop the unwanted column
fr_cities = fr_cities.drop(['Rang2021','Statut','id'],axis=1)

print (" I have a list of " + str(fr_cities.shape[0]) + " cities, I need only the biggest cities per departement.")
print (" This is concerning " + str(len(fr_cities['Département'].unique())) + " departement.")

# keep only the largest city for each department :
# by doing: sort by Population légale and drop duplicate keep only first occurence :
tmp = fr_cities.sort_values(by=['Population légale'],ascending=False)
fr_cities = tmp.drop_duplicates(subset=['Département'], keep='first')

print (" I have now a reduced list of " + str(fr_cities.shape[0]) + " cities, keeping largest per department.")
print (" This is concerning " + str(len(fr_cities['Département'].unique())) + " departement.")
display (fr_cities.tail())


fr_cities_departement = fr_cities['Département'].unique()


Merge two data sets: largest cities and list of cities with their coordinates ( based on Insee Code)
 I have a list of 278 cities, I need only the biggest cities per departement.
 This is concerning 86 departement.
 I have now a reduced list of 86 cities, keeping largest per department.
 This is concerning 86 departement.


,insee_code,Commune,Département,Région,Population légale,department_code,zip_code,name,slug,gps_lat,gps_lng
372,89024,Auxerre,Yonne,Bourgogne-Franche-Comté,34764,89,89000,Auxerre,auxerre,47.797359,3.565950
385,58194,Nevers,Nièvre,Bourgogne-Franche-Comté,33279,58,58000,Nevers,nevers,46.988185,3.160122
390,47001,Agen,Lot-et-Garonne,Nouvelle-Aquitaine,33012,47,47000,Agen,agen,44.207674,0.630228
399,88160,Épinal,Vosges,Grand Est,32223,88,88000,Épinal,epinal,48.166886,6.474717
424,24322,Périgueux,Dordogne,Nouvelle-Aquitaine,30060,24,24000,Périgueux,perigueux,45.194904,0.709100


In [12]:
## What departement are we missing : i.e where there is no "big cities". These departement cannot be included in this study as there is no official data source found for them
# Relation between Insee Code and Departement code is : Departement are the first digit of the INSEE code . e.g all Insee code starting with 06xxxx relates to 06 Departement code
# from this , search what INSEE code is missing in the big cities dataset
departement_big_french_cities = big_french_cities.insee_code.astype(str).str[0:2].unique()

display (HTML(" Departement which will are considered : <br> Others - as we don't have big city information on them - are not going to be included "))
print ([x for x in fr_cities_departement if x not in departement_big_french_cities])


['Paris', 'Bouches-du-Rhône', 'Métropole de Lyon[Note 1]', 'Haute-Garonne', 'Alpes-Maritimes', 'Loire-Atlantique', 'Hérault', 'Bas-Rhin', 'Gironde', 'Nord', 'Ille-et-Vilaine', 'Marne', 'Var', 'Loire', 'Seine-Maritime', 'Isère', "Côte-d'Or", 'Maine-et-Loire', 'La Réunion', 'Gard', 'Puy-de-Dôme', 'Sarthe', 'Finistère', 'Indre-et-Loire', 'Somme', 'Haute-Vienne', 'Haute-Savoie', 'Hauts-de-Seine', 'Pyrénées-Orientales', 'Doubs', 'Moselle', 'Loiret', 'Seine-Saint-Denis', "Val-d'Oise", 'Haut-Rhin', 'Calvados', 'Meurthe-et-Moselle', 'Val-de-Marne', 'Vaucluse', 'Vienne', 'Yvelines', 'Manche', 'Martinique', 'Pyrénées-Atlantiques', 'Charente-Maritime', 'Pas-de-Calais', 'Mayotte', 'Corse-du-Sud', 'Essonne', 'Drôme', 'Cher', 'Guyane', 'Aube', 'Tarn-et-Garonne', 'Deux-Sèvres', 'Savoie', 'Morbihan', 'Oise', 'Seine-et-Marne', 'Aude', 'Vendée', 'Aisne', 'Guadeloupe', 'Mayenne', 'Tarn', 'Haute-Corse', 'Territoire de Belfort', 'Eure', 'Corrèze', 'Ardennes', 'Loir-et-Cher', 'Saône-et-Loire', "Côtes-d'Armo

## Gather the famous social places for each of our  largest cities in France

In [16]:
## Initialize the Foursquare data:
CLIENT_ID = '3S1PFTNL1HAFWEYMYHRLCVZVWN1DSK0IPHKSX5JXAGR3E1ML' # your Foursquare ID
CLIENT_SECRET = 'V4IEPFK4NJZZER4LJA33R2LX3S5KRBHVECAFZJDVTLFZUTBP' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 80
# function to get the venues for each cities 
# cities are taken from fr_cities , based on gps_lat,gps_lng for each Commune
def getNearbyVenues(names, latitudes, longitudes, radius=20000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results ])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Commune', 
                  'Commune Latitude', 
                  'Commune Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    ## filter out the Venues category we don't want .. we want to keep only what we defined as social places in the below list
    
    ## JSA update 27 Dec : make the same study without filters on social places to see whether cluster observation are differents 
    ### JSA update 27 Dec social_places = ['Restaurant','Coffee','Café','Tea Room','Bar','Bistro','Cinema']
    ### JSA update 27 Dec nearby_venues = nearby_venues.loc[nearby_venues['Venue Category'].str.contains('|'.join(social_places))]
    
    return(nearby_venues)


# Call the function for all our Cities and get back a dataframe with all the Venues
fr_cities_venues = getNearbyVenues(names=fr_cities['Commune'],
                                   latitudes=fr_cities['gps_lat'],
                                   longitudes=fr_cities['gps_lng']
                                  )

display (fr_cities_venues.head())

Paris
https://api.foursquare.com/v2/venues/explore?&client_id=3S1PFTNL1HAFWEYMYHRLCVZVWN1DSK0IPHKSX5JXAGR3E1ML&client_secret=V4IEPFK4NJZZER4LJA33R2LX3S5KRBHVECAFZJDVTLFZUTBP&v=20180604&ll=48.8640493,2.3310526&radius=20000&limit=80
Marseille
https://api.foursquare.com/v2/venues/explore?&client_id=3S1PFTNL1HAFWEYMYHRLCVZVWN1DSK0IPHKSX5JXAGR3E1ML&client_secret=V4IEPFK4NJZZER4LJA33R2LX3S5KRBHVECAFZJDVTLFZUTBP&v=20180604&ll=43.300771000000005,5.3821768&radius=20000&limit=80
Lyon
https://api.foursquare.com/v2/venues/explore?&client_id=3S1PFTNL1HAFWEYMYHRLCVZVWN1DSK0IPHKSX5JXAGR3E1ML&client_secret=V4IEPFK4NJZZER4LJA33R2LX3S5KRBHVECAFZJDVTLFZUTBP&v=20180604&ll=45.77129179999999,4.8280831&radius=20000&limit=80
Toulouse
https://api.foursquare.com/v2/venues/explore?&client_id=3S1PFTNL1HAFWEYMYHRLCVZVWN1DSK0IPHKSX5JXAGR3E1ML&client_secret=V4IEPFK4NJZZER4LJA33R2LX3S5KRBHVECAFZJDVTLFZUTBP&v=20180604&ll=43.6046256,1.444205&radius=20000&limit=80
Nice
https://api.foursquare.com/v2/venues/explore?&clien

Caen
https://api.foursquare.com/v2/venues/explore?&client_id=3S1PFTNL1HAFWEYMYHRLCVZVWN1DSK0IPHKSX5JXAGR3E1ML&client_secret=V4IEPFK4NJZZER4LJA33R2LX3S5KRBHVECAFZJDVTLFZUTBP&v=20180604&ll=49.181788058824,-0.37257017647059&radius=20000&limit=80
Nancy
https://api.foursquare.com/v2/venues/explore?&client_id=3S1PFTNL1HAFWEYMYHRLCVZVWN1DSK0IPHKSX5JXAGR3E1ML&client_secret=V4IEPFK4NJZZER4LJA33R2LX3S5KRBHVECAFZJDVTLFZUTBP&v=20180604&ll=48.689645899999995,6.1737197&radius=20000&limit=80
Vitry-sur-Seine
https://api.foursquare.com/v2/venues/explore?&client_id=3S1PFTNL1HAFWEYMYHRLCVZVWN1DSK0IPHKSX5JXAGR3E1ML&client_secret=V4IEPFK4NJZZER4LJA33R2LX3S5KRBHVECAFZJDVTLFZUTBP&v=20180604&ll=48.790497435896995,2.3905555128205003&radius=20000&limit=80
Avignon
https://api.foursquare.com/v2/venues/explore?&client_id=3S1PFTNL1HAFWEYMYHRLCVZVWN1DSK0IPHKSX5JXAGR3E1ML&client_secret=V4IEPFK4NJZZER4LJA33R2LX3S5KRBHVECAFZJDVTLFZUTBP&v=20180604&ll=43.9470042,4.8209596&radius=20000&limit=80
Poitiers
https://api.foursq

Charleville-Mézières
https://api.foursquare.com/v2/venues/explore?&client_id=3S1PFTNL1HAFWEYMYHRLCVZVWN1DSK0IPHKSX5JXAGR3E1ML&client_secret=V4IEPFK4NJZZER4LJA33R2LX3S5KRBHVECAFZJDVTLFZUTBP&v=20180604&ll=49.775825972222,4.720758125&radius=20000&limit=80
Blois
https://api.foursquare.com/v2/venues/explore?&client_id=3S1PFTNL1HAFWEYMYHRLCVZVWN1DSK0IPHKSX5JXAGR3E1ML&client_secret=V4IEPFK4NJZZER4LJA33R2LX3S5KRBHVECAFZJDVTLFZUTBP&v=20180604&ll=47.583407047619,1.3182003809523999&radius=20000&limit=80
Chalon-sur-Saône
https://api.foursquare.com/v2/venues/explore?&client_id=3S1PFTNL1HAFWEYMYHRLCVZVWN1DSK0IPHKSX5JXAGR3E1ML&client_secret=V4IEPFK4NJZZER4LJA33R2LX3S5KRBHVECAFZJDVTLFZUTBP&v=20180604&ll=46.78945592233,4.8440952427185&radius=20000&limit=80
Saint-Brieuc
https://api.foursquare.com/v2/venues/explore?&client_id=3S1PFTNL1HAFWEYMYHRLCVZVWN1DSK0IPHKSX5JXAGR3E1ML&client_secret=V4IEPFK4NJZZER4LJA33R2LX3S5KRBHVECAFZJDVTLFZUTBP&v=20180604&ll=48.515869140893,-2.7629354295532997&radius=20000&limit=

,Commune,Commune Latitude,Commune Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Paris,48.864049,2.331053,Jardin des Tuileries,48.863642,2.326484,Garden
1,Paris,48.864049,2.331053,Jardin du Palais Royal,48.864941,2.337728,Garden
2,Paris,48.864049,2.331053,Hôtel Ritz,48.868145,2.328962,Hotel
3,Paris,48.864049,2.331053,Musée d'Orsay,48.859993,2.326387,Art Museum
4,Paris,48.864049,2.331053,Place Vendôme,48.867798,2.329741,Plaza


###  I'm now having the count of social places within the given radius of my biggest cities. 

data set <b>fr_cities_venues_grouped</b> is containing the list of Commune, with their coordinates and the number of Social places found

In [17]:
#print ("What are the venue category we have " )
#print (fr_cities_venues['Venue Category'].unique())

# Count Social Venues per City
print ("Number of venues per largest city in each departement  " )
fr_cities_venues_grouped = fr_cities_venues.groupby(['Commune','Commune Latitude','Commune Longitude'])['Venue'].count().reset_index()
display (fr_cities_venues_grouped)
# Count per Venue category
print ("Count of venues per largest city in each departement - group by category " )
display (fr_cities_venues.groupby(['Commune','Commune Latitude','Commune Longitude','Venue Category']).count())


Number of venues per largest city in each departement  


,Commune,Commune Latitude,Commune Longitude,Venue
0,Agen,44.207674,0.630228,36
1,Ajaccio,41.918489,8.671060,76
2,Albi,43.918466,2.145517,34
3,Amiens,49.899050,2.275277,50
4,Angers,47.501835,-0.517146,70
5,Angoulême,45.647789,0.151222,47
6,Annecy,45.897172,6.125161,52
7,Argenteuil,48.954656,2.243380,80
8,Auxerre,47.797359,3.565950,40
9,Avignon,43.947004,4.820960,80


Count of venues per largest city in each departement - group by category 


Venue  \
Commune                    Commune Latitude Commune Longitude Venue Category                      
Agen                       44.207674        0.630228          BBQ Joint                       1   
                                                              Bar                             1   
                                                              Bookstore                       1   
                                                              Breakfast Spot                  1   
                                                              Cosmetics Shop                  1   
                                                              Department Store                1   
                                                              Diner                           1   
                                                              Farm                            1   
                                                              Farmers Market                  1   
                                                              Fast Food Restaurant            3   
                                                              French Restaurant               3   
                                                              Hotel                           1   
                                                              Japanese Restaurant             1   
                                                              Mobile Phone Shop               1   
                                                              Multiplex                       2   
                                                              Photography Studio              1   
                                                              Plaza                           1   
                                                              Rugby Pitch                     1   
                                                              Sandwich Place                  1   
                                                              Supermarket                     8   
                                                              Theme Park                      1   
                                                              Train Station                   3   
Ajaccio                    41.918489        8.671060          Art Museum                      1   
                                                              Bar                             3   
                                                              Beach                          14   
                                                              Beach Bar                       1   
                                                              Bistro                          1   
                                                              Café                            2   
                                                              Construction & Landscaping      1   
                                                              Corsican Restaurant             3   
...                                                                                         ...   
Évry-Courcouronnes[Note 4] 48.628839        2.439091          French Restaurant               2   
                                                              Furniture / Home Store          3   
                                                              Go Kart Track                   1   
                                                              Golf Course                     1   
                                                              Grocery Store                   1   
                                                              Gym                             1   
                                                              Hotel                           1   
                                                              Indian Restaurant               3   
                               

## Include  Covid data and merge them with the Cities data
#### To be able to have a visual representation of the Covid data, I'm going to use the largest City of each department , as the Covid information are based on department. 
#### This is the full data set I'm going to use for clustering analysis

In [18]:
#display (covid_data.head())
#display (fr_cities.head())

print ("Merge two data sets: Cities data set and Covid data( based on Department 2 digits number )" )
## covid_data => dep & fr_cities => Département
# merge our datafiles with the coordinate dataframe
covid_city_data = pd.merge(covid_data_refine, fr_cities, left_on='dep', right_on='department_code', how='inner') # Join info Departement info

#display (covid_city_data)

## I then keep only the largest City of each department as representative data
myminpop = 10000
print ("covid_city_data before drop ",covid_city_data.shape)
covid_city_data.drop(covid_city_data[covid_city_data['Population légale'] < myminpop].index, inplace=True)
print ("covid_city_data AFTER drop ",covid_city_data.shape)
display (covid_city_data.head())

Merge two data sets: Cities data set and Covid data( based on Department 2 digits number )
covid_city_data before drop  (86, 13)
covid_city_data AFTER drop  (86, 13)


,dep,PtoT_ratio,insee_code,Commune,Département,Région,Population légale,department_code,zip_code,name,slug,gps_lat,gps_lng
0,01,0.067749,01053,Bourg-en-Bresse,Ain,Auvergne-Rhône-Alpes,41248,01,1000,Bourg-en-Bresse,bourg en bresse,46.207450,5.246124
1,02,0.052157,02691,Saint-Quentin,Aisne,Hauts-de-France,53856,02,2100,Saint-Quentin,saint quentin,49.846348,3.288463
2,03,0.041838,03185,Montluçon,Allier,Auvergne-Rhône-Alpes,34938,03,3100,Montluçon,montlucon,46.341504,2.601539
3,05,0.060741,05061,Gap,Hautes-Alpes,Provence-Alpes-Côte d'Azur,40559,05,5000,Gap,gap,44.576617,6.060768
4,06,0.042116,06088,Nice,Alpes-Maritimes,Provence-Alpes-Côte d'Azur,341032,06,6000,Nice,nice,43.696036,7.265590


## Final data set build : to have all the data on a single data set:  Covid data , Cities and Venues informations
merge the fr_cities_venues_grouped with covid_city_data on the Commune name

In [19]:
#print (fr_cities_venues_grouped.dtypes)
#print (covid_city_data.dtypes)

final_dataset = pd.merge(fr_cities_venues_grouped, covid_city_data, on='Commune', how='inner') # Join info Departement info
# cast the output format the covid data to keep less float digits 
pd.options.display.float_format = '{:,.4f}'.format

print (final_dataset.shape)
display (final_dataset)

#print (final_dataset.dtypes)

(88, 16)


,Commune,Commune Latitude,Commune Longitude,Venue,dep,PtoT_ratio,insee_code,Département,Région,Population légale,department_code,zip_code,name,slug,gps_lat,gps_lng
0,Agen,44.2077,0.6302,36,47,0.0440,47001,Lot-et-Garonne,Nouvelle-Aquitaine,33012,47,"47,000.0000",Agen,agen,44.2077,0.6302
1,Ajaccio,41.9185,8.6711,76,2A,0.0222,2A004,Corse-du-Sud,Corse,70817,2A,"20,000.0000",Ajaccio,ajaccio,41.9185,8.6711
2,Albi,43.9185,2.1455,34,81,0.0454,81004,Tarn,Occitanie,48993,81,"81,000.0000",Albi,albi,43.9185,2.1455
3,Amiens,49.8991,2.2753,50,80,0.0400,80021,Somme,Hauts-de-France,133891,80,"80,000.0000",Amiens,amiens,49.8991,2.2753
4,Angers,47.5018,-0.5171,70,49,0.0409,49007,Maine-et-Loire,Pays de la Loire,154508,49,"49,000.0000",Angers,angers,47.5018,-0.5171
5,Angoulême,45.6478,0.1512,47,16,0.0314,16015,Charente,Nouvelle-Aquitaine,41711,16,"16,000.0000",Angoulême,angouleme,45.6478,0.1512
6,Annecy,45.8972,6.1252,52,74,0.0745,74010,Haute-Savoie,Auvergne-Rhône-Alpes,128199,74,"74,000.0000",Annecy,annecy,45.8972,6.1252
7,Argenteuil,48.9547,2.2434,80,95,0.0655,95018,Val-d'Oise,Île-de-France,110213,95,"95,100.0000",Argenteuil,argenteuil,48.9547,2.2434
8,Auxerre,47.7974,3.5660,40,89,0.0480,89024,Yonne,Bourgogne-Franche-Comté,34764,89,"89,000.0000",Auxerre,auxerre,47.7974,3.5660
9,Avignon,43.9470,4.8210,80,84,0.0545,84007,Vaucluse,Provence-Alpes-Côte d'Azur,91729,84,"84,000.0000",Avignon,avignon,43.9470,4.8210


## display Covid data on a map to get a first sight

### The larger is the point on the map  the higher is the Postive/Test ratio ( Ratio built on mean value of all the Covid data available)


In [20]:
# create map of New York using latitude and longitude values
latitude = 46.156782
longitude = 4.924699
map_france = folium.Map(location=[latitude, longitude], zoom_start=7) ## map centered on Paris coordinates

# add markers to map of France
for lat, lng, cityname,zip_code,covid in zip(final_dataset['gps_lat'], final_dataset['gps_lng'], final_dataset['Commune'], final_dataset['zip_code'],final_dataset['PtoT_ratio']):
    label = '{}, {}, {:,.3f}'.format(int(zip_code), cityname,covid)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=200*covid,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_france)
    
map_france


## Work with data set ( covid + cities pop + coordinates ) for clustering 
#### Clustering will be based on Cities having the most ptoPop_ratio and their social places 


In [21]:
display (HTML('Normalize (OneHot encoding) the fr_cities_venues data to prepare for Kmean'))

# one hot encoding
fr_cities_venues_onehot = pd.get_dummies(fr_cities_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
fr_cities_venues_onehot['Commune'] = fr_cities_venues['Commune'] 

# move neighborhood column to the first column
fixed_columns = [fr_cities_venues_onehot.columns[-1]] + list(fr_cities_venues_onehot.columns[:-1])
fr_cities_venues_onehot = fr_cities_venues_onehot[fixed_columns]

#display (fr_cities_venues_onehot.head(25))
fr_cities_venues_onehot_grouped = fr_cities_venues_onehot.groupby('Commune').mean().reset_index()
fr_cities_venues_onehot_grouped.head(25)

,Commune,Airport,Alsatian Restaurant,American Restaurant,Apres Ski Bar,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,...,Vineyard,Warehouse Store,Water Park,Waterfall,Waterfront,Wine Bar,Wine Shop,Winery,Zoo,Zoo Exhibit
0,Agen,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,Ajaccio,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0132,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2,Albi,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0294,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0294,0.0000,0.0000,0.0000,0.0000
3,Amiens,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0200,0.0000
4,Angers,0.0143,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0286,0.0000,...,0.0000,0.0000,0.0143,0.0000,0.0000,0.0143,0.0143,0.0000,0.0000,0.0000
5,Angoulême,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0213,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
6,Annecy,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0192,0.0000,0.0000,0.0000,0.0000
7,Argenteuil,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0500,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
8,Auxerre,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0250,0.0000,0.0000,0.0000,0.0000,0.0000,0.0250,0.0250,0.0000,0.0000
9,Avignon,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0125,0.0000,...,0.0250,0.0000,0.0125,0.0000,0.0000,0.0250,0.0125,0.0000,0.0000,0.0000


In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Commune']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
fr_cities_venues_sorted = pd.DataFrame(columns=columns)
fr_cities_venues_sorted['Commune'] = fr_cities_venues_onehot_grouped['Commune']

for ind in np.arange(fr_cities_venues_onehot_grouped.shape[0]):
    fr_cities_venues_sorted.iloc[ind, 1:] = return_most_common_venues(fr_cities_venues_onehot_grouped.iloc[ind, :], num_top_venues)

fr_cities_venues_sorted

,Commune,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agen,Supermarket,Train Station,Fast Food Restaurant,French Restaurant,Multiplex,Farmers Market,Farm,Mobile Phone Shop,Theme Park,Diner
1,Ajaccio,Beach,Hotel,French Restaurant,Corsican Restaurant,Bar,Plaza,Seafood Restaurant,Restaurant,Pizza Place,Italian Restaurant
2,Albi,Train Station,French Restaurant,Supermarket,Shopping Mall,Fast Food Restaurant,Hotel,Restaurant,Steakhouse,Cosmetics Shop,Sporting Goods Shop
3,Amiens,Train Station,French Restaurant,Plaza,Bar,Supermarket,Fast Food Restaurant,Performing Arts Venue,Park,Coffee Shop,Soccer Stadium
4,Angers,Train Station,Supermarket,Park,Bar,French Restaurant,Castle,Hotel,Café,Indian Restaurant,Fast Food Restaurant
5,Angoulême,Hotel,Supermarket,Fast Food Restaurant,Steakhouse,Shopping Mall,French Restaurant,Sporting Goods Shop,Farmers Market,Construction & Landscaping,Pub
6,Annecy,Beach,Supermarket,French Restaurant,Ice Cream Shop,Café,Bar,Hotel,Ski Area,Trail,Mountain
7,Argenteuil,Plaza,Hotel,Garden,Art Museum,Historic Site,French Restaurant,Gourmet Shop,Park,Bistro,Sandwich Place
8,Auxerre,Hotel,Train Station,French Restaurant,Restaurant,Supermarket,Sandwich Place,Bar,Fast Food Restaurant,Sporting Goods Shop,Shopping Mall
9,Avignon,French Restaurant,Hotel,Plaza,Historic Site,Italian Restaurant,Ice Cream Shop,Restaurant,Wine Bar,Farmers Market,Coffee Shop


In [23]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

display (HTML('<h3>Run _k_-means to cluster the Commune into N clusters.</h3>'))

# set number of clusters
kclusters = 5

fr_cities_grouped_clustering = fr_cities_venues_onehot_grouped.drop('Commune', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(fr_cities_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 

array([4, 3, 0, 0, 0, 4, 3, 1, 0, 2, 3, 0, 0, 0, 2, 1, 1, 0, 4, 3, 0, 3,
       2, 3, 2, 0, 4, 0, 4, 4, 0, 1, 3, 4, 2, 0, 2, 0, 2, 0, 3, 2, 0, 3,
       1, 3, 2, 3, 0, 4, 4, 3, 2, 0, 2, 4, 4, 3, 0, 2, 2, 1, 2, 3, 0, 0,
       2, 3, 3, 1, 4, 0, 2, 4, 3, 2, 0, 3, 4, 1, 0, 1, 0, 4, 3])

In [35]:
# add clustering labels
try:
    fr_cities_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
except ValueError:
    print (" cluster labels already there you need to recreate the DF fr_cities_venues_sorted above !!  ")


#fr_cities_merged = final_dataset[['Commune','Commune Latitude','Commune Longitude','Venue','pop','PtoT_ratio','dep']]
fr_cities_merged = final_dataset[['Commune','Commune Latitude','Commune Longitude','Venue','PtoT_ratio','dep','Population légale']]


# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
fr_cities_merged = fr_cities_merged.join(fr_cities_venues_sorted.set_index('Commune'), on='Commune')

fr_cities_merged # check the last columns!


 cluster labels already there you need to recreate the DF fr_cities_venues_sorted above !!  


,Commune,Commune Latitude,Commune Longitude,Venue,PtoT_ratio,dep,Population légale,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agen,44.2077,0.6302,36,0.0440,47,33012,4,Supermarket,Train Station,Fast Food Restaurant,French Restaurant,Multiplex,Farmers Market,Farm,Mobile Phone Shop,Theme Park,Diner
1,Ajaccio,41.9185,8.6711,76,0.0222,2A,70817,3,Beach,Hotel,French Restaurant,Corsican Restaurant,Bar,Plaza,Seafood Restaurant,Restaurant,Pizza Place,Italian Restaurant
2,Albi,43.9185,2.1455,34,0.0454,81,48993,0,Train Station,French Restaurant,Supermarket,Shopping Mall,Fast Food Restaurant,Hotel,Restaurant,Steakhouse,Cosmetics Shop,Sporting Goods Shop
3,Amiens,49.8991,2.2753,50,0.0400,80,133891,0,Train Station,French Restaurant,Plaza,Bar,Supermarket,Fast Food Restaurant,Performing Arts Venue,Park,Coffee Shop,Soccer Stadium
4,Angers,47.5018,-0.5171,70,0.0409,49,154508,0,Train Station,Supermarket,Park,Bar,French Restaurant,Castle,Hotel,Café,Indian Restaurant,Fast Food Restaurant
5,Angoulême,45.6478,0.1512,47,0.0314,16,41711,4,Hotel,Supermarket,Fast Food Restaurant,Steakhouse,Shopping Mall,French Restaurant,Sporting Goods Shop,Farmers Market,Construction & Landscaping,Pub
6,Annecy,45.8972,6.1252,52,0.0745,74,128199,3,Beach,Supermarket,French Restaurant,Ice Cream Shop,Café,Bar,Hotel,Ski Area,Trail,Mountain
7,Argenteuil,48.9547,2.2434,80,0.0655,95,110213,1,Plaza,Hotel,Garden,Art Museum,Historic Site,French Restaurant,Gourmet Shop,Park,Bistro,Sandwich Place
8,Auxerre,47.7974,3.5660,40,0.0480,89,34764,0,Hotel,Train Station,French Restaurant,Restaurant,Supermarket,Sandwich Place,Bar,Fast Food Restaurant,Sporting Goods Shop,Shopping Mall
9,Avignon,43.9470,4.8210,80,0.0545,84,91729,2,French Restaurant,Hotel,Plaza,Historic Site,Italian Restaurant,Ice Cream Shop,Restaurant,Wine Bar,Farmers Market,Coffee Shop


In [25]:
display (HTML('<h3>Visual the cluster in a map.</h3>'))

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=7)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster, covid in zip(fr_cities_merged['Commune Latitude'], fr_cities_merged['Commune Longitude'], fr_cities_merged['Commune'], fr_cities_merged['Cluster Labels'], fr_cities_merged['PtoT_ratio']):
    label = '{}, Cluster {}, PtoT {:,.3f}'.format(str(poi), cluster,covid)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=200*covid,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters 

#### The clustering groups the largest French cities into 5 clusters (I set it ot 5), based on the most visited places, here grouped by categories. 
<br> - There is no region specific grouping in the observation we are making . Cluster members are from all over the French Territory ( including those out of metropole , in the Pacific ocean and Atlanctic where we have Fench's Islands 
<br> - Initial obeservation shows that there is not big difference in the "positive to tested for Covid" ratio, which is from  4.2 to 5.1% 
<br> - Per Cluster observation shows that there are cities where there is a bigger "positive to tested for Covid" ratio, above the max of the average ( so > 5.1% ) .. Thse cities are spreaded across different regions, belonging to different clusters. 
However, 4 out of the 16 Cities beyond 5.1% are belonging to the same region : Wider Paris Cities, and they have in common some most visited places : Hotel, Supermarket, Restaurant and Transportation areas (Train or Airport)



In [72]:
display (fr_cities_merged.loc[fr_cities_merged['PtoT_ratio'] > 0.06].sort_values(by=['PtoT_ratio'],ascending=False))

,Commune,Commune Latitude,Commune Longitude,Venue,PtoT_ratio,dep,Population légale,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Cayenne,4.9808,-52.3235,20,0.0921,973,63652,3,Shopping Mall,Airport,Pizza Place,Big Box Store,Supermarket,Resort,Rental Car Location,Coffee Shop,Park,Bar
45,Mamoudzou,-12.7898,45.2253,4,0.0851,976,71437,3,Café,Toy / Game Store,Board Shop,Resort,Falafel Restaurant,Dive Bar,Drugstore,Duty-free Shop,Electronics Store,Estaminet
6,Annecy,45.8972,6.1252,52,0.0745,74,128199,3,Beach,Supermarket,French Restaurant,Ice Cream Shop,Café,Bar,Hotel,Ski Area,Trail,Mountain
34,Grenoble,45.1894,5.7165,80,0.0725,38,157650,2,French Restaurant,Supermarket,Train Station,Ski Area,Plaza,Pub,Restaurant,Farmers Market,Hotel,Tea Room
25,Chambéry,45.5800,5.9129,49,0.0718,73,58833,0,Train Station,Hotel,French Restaurant,Park,Shopping Mall,Multiplex,Fast Food Restaurant,Beach,Supermarket,Resort
69,Saint-Denis,-20.9445,55.4489,46,0.0696,93,112091,1,Hotel,French Restaurant,Shopping Mall,Plaza,Bakery,Park,Farmers Market,Historic Site,Art Museum,Garden
71,Saint-Denis,48.9316,2.3563,80,0.0696,93,112091,1,Hotel,French Restaurant,Shopping Mall,Plaza,Bakery,Park,Farmers Market,Historic Site,Art Museum,Garden
44,Lyon,45.7713,4.8281,80,0.0692,69,518635,1,Plaza,French Restaurant,Café,Hotel,Lyonese Bouchon,Bar,Park,Bakery,Farmers Market,Wine Bar
83,Villefranche-sur-Saône,45.9875,4.7209,52,0.0692,69,36288,0,Train Station,Hotel,French Restaurant,Restaurant,Fast Food Restaurant,Supermarket,Rest Area,Diner,Gas Station,Cafeteria
74,Saint-Étienne,45.4506,4.3859,67,0.0682,42,173089,0,Train Station,Hotel,Supermarket,Plaza,French Restaurant,Fast Food Restaurant,Coffee Shop,Steakhouse,Pub,Furniture / Home Store


In [76]:

display (HTML('<h3>Cluster 0 -- with '+str(int(fr_cities_merged.loc[fr_cities_merged['Cluster Labels'] == 0].agg({'Population légale':'sum'})))+' peoples, "Positive over tested case mean" in that cluster: '+str(float(fr_cities_merged.loc[fr_cities_merged['Cluster Labels'] == 0].agg({'PtoT_ratio':'mean'})))+'<p style="color:Red;">Red</p> </h3>'))
display ('In that cluster it is clear that Train stations/Transportation as well as restaurant are the most visited places and transportation are usually closed areas.. Cluster 0 is composed of largest cities where there is a lot of public transportation, eg bus or train. It would make sens to be very caution on how we handle public transport when thinking of Covid19 spread.. e.g before closing shops or libraries')

display (HTML('<h3>Cluster 1 -- with '+str(int(fr_cities_merged.loc[fr_cities_merged['Cluster Labels'] == 1].agg({'Population légale':'sum'})))+' peoples, "Positive over tested case" mean in that cluster: '+str(float(fr_cities_merged.loc[fr_cities_merged['Cluster Labels'] == 1].agg({'PtoT_ratio':'mean'})))+' <p style="color:Purple;">Purple</p> </h3>'))
display ('In that cluster we observe that the common most visited places are Hotel and Plaza (outside famous places).. This may suggest that hotel could be an significant vector of Covid19 propagation in these cities. thisi is also the cluster showing the highest Covid 19 Positive ratio ')

display (HTML('<h3>Cluster 2 -- with '+str(int(fr_cities_merged.loc[fr_cities_merged['Cluster Labels'] == 2].agg({'Population légale':'sum'})))+' peoples, "Positive over tested case" mean in that cluster: '+str(float(fr_cities_merged.loc[fr_cities_merged['Cluster Labels'] == 2].agg({'PtoT_ratio':'mean'})))+' <p style="color:blue;">Blue</p> </h3>'))
display ('In that cluster we observe that clearly the restaurant are the top visited place. It suggest that in these cities is may be relevant to work on Restaurant area to reduce the virus spread. Other places/shops not being from the top10 observed could remain open')

display (HTML('<h3>Cluster 3 -- with '+str(int(fr_cities_merged.loc[fr_cities_merged['Cluster Labels'] == 3].agg({'Population légale':'sum'})))+' peoples, "Positive over tested case" mean in that cluster: '+str(float(fr_cities_merged.loc[fr_cities_merged['Cluster Labels'] == 3].agg({'PtoT_ratio':'mean'})))+' <p style="color:Green;">Light Green</p> </h3>'))
display ('In that cluster we observe that REstaurant, Hotel and Café are the most visited areas. Recommendations for these cities could be to adapted and focused on these areas')

display (HTML('<h3>Cluster 4 -- with '+str(int(fr_cities_merged.loc[fr_cities_merged['Cluster Labels'] == 4].agg({'Population légale':'sum'})))+' peoples, "Positive over tested case" mean in that cluster: '+str(float(fr_cities_merged.loc[fr_cities_merged['Cluster Labels'] == 4].agg({'PtoT_ratio':'mean'})))+' <p style="color:Orange;">Orange</p> </h3>'))
display ('From that cluster we observe that Hotel and Supermarkets are the most visited areas. Although Hotel lock down could be handled, Supermarket is hard to handle as it is part of the primary shops to remains open. Extra sanity care should be done in the supermarket in these cities ')

'In that cluster we observe that Train stations as well as restaurant are the most crowded places.. Cluster 0 is composed of largest cities where there is a lot of public transportation, eg bus or train. It would make sens to be very caution on how we handle public transport when thinking of Covid19 spread.. e.g before closing shops or libraries'

'In that cluster we observe that the common most visited places are Hotel and Plaza (outside famous places).. This may suggest that hotel could be an significant vector of Covid19 propagation in these cities. thisi is also the cluster showing the highest Covid 19 Positive ratio '

'In that cluster we observe that clearly the restaurant are the top visited place. It suggest that in these cities is may be relevant to work on Restaurant area to reduce the virus spread. Other places/shops not being from the top10 observed could remain open'

'In that cluster we observe that REstaurant, Hotel and Café are the most visited areas. Recommendations for these cities could be to adapted and focused on these areas'

'From that cluster we observe that Hotel and Supermarkets are the most visited areas. Although Hotel lock down could be handled, Supermarket is hard to handle as it is part of the primary shops to remains open. Extra sanity care should be done in the supermarket in these cities '

In [78]:
cluster_list = fr_cities_merged['Cluster Labels'].unique()
for cluster in cluster_list:
    display (HTML('<h3>Cluster '+str(cluster)+' with a total population of '+str(int(fr_cities_merged.loc[fr_cities_merged['Cluster Labels'] == cluster].agg({'Population légale':'sum'})))+' peoples </h3>'))
    #display (fr_cities_merged.loc[fr_cities_merged['Cluster Labels'] == cluster].agg({'PtoT_ratio':'mean'}).reset_index())
    display (fr_cities_merged.loc[fr_cities_merged['Cluster Labels'] == cluster])


,Commune,Commune Latitude,Commune Longitude,Venue,PtoT_ratio,dep,Population légale,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agen,44.2077,0.6302,36,0.0440,47,33012,4,Supermarket,Train Station,Fast Food Restaurant,French Restaurant,Multiplex,Farmers Market,Farm,Mobile Phone Shop,Theme Park,Diner
5,Angoulême,45.6478,0.1512,47,0.0314,16,41711,4,Hotel,Supermarket,Fast Food Restaurant,Steakhouse,Shopping Mall,French Restaurant,Sporting Goods Shop,Farmers Market,Construction & Landscaping,Pub
18,Bourges,47.0798,2.3951,50,0.0459,18,64668,4,Hotel,Train Station,Supermarket,Fast Food Restaurant,Pub,Italian Restaurant,Concert Hall,Bar,Steakhouse,Shopping Mall
26,Charleville-Mézières,49.7758,4.7208,53,0.0450,08,46391,4,Hotel,Supermarket,Train Station,Fast Food Restaurant,French Restaurant,Bistro,Castle,Bed & Breakfast,Italian Restaurant,Scenic Lookout
28,Cherbourg-en-Cotentin[Note 3],49.6340,-1.6134,41,0.0264,50,79144,4,Supermarket,Hotel,Fast Food Restaurant,Plaza,Construction & Landscaping,Train Station,Seafood Restaurant,Lounge,Museum,French Restaurant
29,Châteauroux,46.8062,1.6913,27,0.0370,36,43442,4,Hotel,Supermarket,Restaurant,Fast Food Restaurant,Café,French Restaurant,Train Station,Rest Area,Automotive Shop,Gas Station
33,Gap,44.5766,6.0608,35,0.0607,05,40559,4,French Restaurant,Supermarket,Hotel,Apres Ski Bar,Diner,Shopping Mall,Ski Area,Train Station,Resort,Café
49,Montauban,44.0225,1.3717,28,0.0408,82,60952,4,Supermarket,Fast Food Restaurant,Hotel,Train Station,Department Store,Movie Theater,Restaurant,Steakhouse,French Restaurant,Shop & Service
50,Montluçon,46.3415,2.6015,15,0.0418,03,34938,4,Fast Food Restaurant,Supermarket,Hotel,Rest Area,Automotive Shop,Steakhouse,Sporting Goods Shop,Perfume Shop,Train Station,Electronics Store
55,Narbonne,43.1649,3.0302,76,0.0292,11,55375,4,Fast Food Restaurant,Supermarket,French Restaurant,Train Station,Hotel,Beach,Historic Site,Harbor / Marina,Restaurant,Bar


,Commune,Commune Latitude,Commune Longitude,Venue,PtoT_ratio,dep,Population légale,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Ajaccio,41.9185,8.6711,76,0.0222,2A,70817,3,Beach,Hotel,French Restaurant,Corsican Restaurant,Bar,Plaza,Seafood Restaurant,Restaurant,Pizza Place,Italian Restaurant
6,Annecy,45.8972,6.1252,52,0.0745,74,128199,3,Beach,Supermarket,French Restaurant,Ice Cream Shop,Café,Bar,Hotel,Ski Area,Trail,Mountain
10,Bastia,42.7381,9.4228,63,0.0224,2B,48044,3,Beach,Seafood Restaurant,Hotel,Café,Bar,Shopping Mall,French Restaurant,Plaza,Restaurant,Harbor / Marina
19,Brest,48.4029,-4.5017,73,0.0213,29,139602,3,Supermarket,Bar,Train Station,Creperie,Fast Food Restaurant,French Restaurant,Beach,Hotel,Department Store,Scenic Lookout
21,Caen,49.1818,-0.3726,80,0.0323,14,105512,3,Beach,Hotel,Supermarket,History Museum,Bar,French Restaurant,Fast Food Restaurant,Multiplex,Movie Theater,Restaurant
23,Cayenne,4.9808,-52.3235,20,0.0921,973,63652,3,Shopping Mall,Airport,Pizza Place,Big Box Store,Supermarket,Resort,Rental Car Location,Coffee Shop,Park,Bar
32,Fort-de-France,14.6415,-61.0685,80,0.0356,972,78126,3,Bakery,Caribbean Restaurant,Beach,Pizza Place,Ice Cream Shop,Hotel,Cocktail Bar,Furniture / Home Store,Distillery,Café
40,Les Abymes,16.2573,-61.5168,80,0.0611,971,53082,3,Beach,Fast Food Restaurant,Nightclub,Hotel,Pizza Place,Caribbean Restaurant,Shopping Mall,Japanese Restaurant,Cocktail Bar,Bakery
43,Lorient,47.7466,-3.3783,60,0.0297,56,57084,3,Supermarket,Bar,Beach,Train Station,Fast Food Restaurant,Clothing Store,Hotel,Harbor / Marina,Department Store,Multiplex
45,Mamoudzou,-12.7898,45.2253,4,0.0851,976,71437,3,Café,Toy / Game Store,Board Shop,Resort,Falafel Restaurant,Dive Bar,Drugstore,Duty-free Shop,Electronics Store,Estaminet


,Commune,Commune Latitude,Commune Longitude,Venue,PtoT_ratio,dep,Population légale,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Albi,43.9185,2.1455,34,0.0454,81,48993,0,Train Station,French Restaurant,Supermarket,Shopping Mall,Fast Food Restaurant,Hotel,Restaurant,Steakhouse,Cosmetics Shop,Sporting Goods Shop
3,Amiens,49.8991,2.2753,50,0.0400,80,133891,0,Train Station,French Restaurant,Plaza,Bar,Supermarket,Fast Food Restaurant,Performing Arts Venue,Park,Coffee Shop,Soccer Stadium
4,Angers,47.5018,-0.5171,70,0.0409,49,154508,0,Train Station,Supermarket,Park,Bar,French Restaurant,Castle,Hotel,Café,Indian Restaurant,Fast Food Restaurant
8,Auxerre,47.7974,3.5660,40,0.0480,89,34764,0,Hotel,Train Station,French Restaurant,Restaurant,Supermarket,Sandwich Place,Bar,Fast Food Restaurant,Sporting Goods Shop,Shopping Mall
11,Beauvais,49.4405,2.0847,41,0.0500,60,56605,0,Train Station,Hotel,Construction & Landscaping,Fast Food Restaurant,Shopping Mall,Sandwich Place,Supermarket,Steakhouse,Diner,Theme Park
12,Belfort,47.6487,6.8376,61,0.0413,90,46954,0,Train Station,Hotel,Fast Food Restaurant,Supermarket,Sporting Goods Shop,Multiplex,Steakhouse,Park,Bar,Museum
13,Besançon,47.2593,6.0272,66,0.0546,25,116775,0,Train Station,Hotel,Plaza,Supermarket,Fast Food Restaurant,Bar,Pub,Restaurant,Farm,Electronics Store
17,Bourg-en-Bresse,46.2074,5.2461,33,0.0677,01,41248,0,Train Station,Hotel,French Restaurant,Fast Food Restaurant,Supermarket,History Museum,Travel & Transport,Convenience Store,Restaurant,Sandwich Place
20,Brive-la-Gaillarde,45.1398,1.5179,30,0.0279,19,46630,0,Train Station,Hotel,Shopping Mall,Supermarket,Fast Food Restaurant,Burger Joint,Football Stadium,Multiplex,Sporting Goods Shop,Snack Place
25,Chambéry,45.5800,5.9129,49,0.0718,73,58833,0,Train Station,Hotel,French Restaurant,Park,Shopping Mall,Multiplex,Fast Food Restaurant,Beach,Supermarket,Resort


,Commune,Commune Latitude,Commune Longitude,Venue,PtoT_ratio,dep,Population légale,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Argenteuil,48.9547,2.2434,80,0.0655,95,110213,1,Plaza,Hotel,Garden,Art Museum,Historic Site,French Restaurant,Gourmet Shop,Park,Bistro,Sandwich Place
15,Bordeaux,44.8350,-0.5873,80,0.0370,33,257068,1,Pedestrian Plaza,Plaza,French Restaurant,Coffee Shop,Bakery,Supermarket,Historic Site,Market,Bar,Shopping Mall
16,Boulogne-Billancourt,48.8345,2.2380,80,0.0499,92,121334,1,Plaza,Garden,Hotel,Historic Site,Park,Botanical Garden,French Restaurant,Art Museum,Gourmet Shop,Electronics Store
31,Dijon,47.3252,5.0357,80,0.0471,21,156854,1,Hotel,Plaza,Train Station,Café,Vineyard,French Restaurant,Fast Food Restaurant,Clothing Store,Shopping Mall,Mobile Phone Shop
44,Lyon,45.7713,4.8281,80,0.0692,69,518635,1,Plaza,French Restaurant,Café,Hotel,Lyonese Bouchon,Bar,Park,Bakery,Farmers Market,Wine Bar
61,Paris,48.8640,2.3311,80,0.0494,75,2175601,1,Plaza,Hotel,Garden,Art Museum,Historic Site,Pastry Shop,Bakery,Sandwich Place,Gourmet Shop,Bookstore
69,Saint-Denis,-20.9445,55.4489,46,0.0696,93,112091,1,Hotel,French Restaurant,Shopping Mall,Plaza,Bakery,Park,Farmers Market,Historic Site,Art Museum,Garden
70,Saint-Denis,-20.9445,55.4489,46,0.0197,974,150535,1,Hotel,French Restaurant,Shopping Mall,Plaza,Bakery,Park,Farmers Market,Historic Site,Art Museum,Garden
71,Saint-Denis,48.9316,2.3563,80,0.0696,93,112091,1,Hotel,French Restaurant,Shopping Mall,Plaza,Bakery,Park,Farmers Market,Historic Site,Art Museum,Garden
72,Saint-Denis,48.9316,2.3563,80,0.0197,974,150535,1,Hotel,French Restaurant,Shopping Mall,Plaza,Bakery,Park,Farmers Market,Historic Site,Art Museum,Garden


,Commune,Commune Latitude,Commune Longitude,Venue,PtoT_ratio,dep,Population légale,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Avignon,43.9470,4.8210,80,0.0545,84,91729,2,French Restaurant,Hotel,Plaza,Historic Site,Italian Restaurant,Ice Cream Shop,Restaurant,Wine Bar,Farmers Market,Coffee Shop
14,Blois,47.5834,1.3182,57,0.0435,41,45871,2,French Restaurant,Supermarket,Hotel,Castle,Train Station,Shopping Mall,Café,Campground,Construction & Landscaping,Restaurant
22,Calais,50.9573,1.8709,80,0.0490,62,72929,2,French Restaurant,Train Station,Hotel,Supermarket,Seafood Restaurant,Beach,Boat or Ferry,Fast Food Restaurant,Brasserie,Clothing Store
24,Chalon-sur-Saône,46.7895,4.8441,22,0.0526,71,44810,2,French Restaurant,Plaza,Train Station,Sporting Goods Shop,Hotel,Cafeteria,Basketball Stadium,Bakery,Rest Area,Shopping Mall
34,Grenoble,45.1894,5.7165,80,0.0725,38,157650,2,French Restaurant,Supermarket,Train Station,Ski Area,Plaza,Pub,Restaurant,Farmers Market,Hotel,Tea Room
36,La Rochelle,46.1575,-1.1693,80,0.0183,17,76114,2,French Restaurant,Seafood Restaurant,Historic Site,Beach,Ice Cream Shop,Plaza,Italian Restaurant,Bar,Hotel,Café
38,Le Havre,49.4770,0.1639,80,0.0494,76,169733,2,French Restaurant,Hotel,Beach,Seafood Restaurant,Café,Bar,Nightclub,Restaurant,Farmers Market,Fish Market
41,Lille,50.6138,3.0424,80,0.0573,59,233098,2,French Restaurant,Bar,Hotel,Clothing Store,Plaza,Art Museum,Bakery,Sporting Goods Shop,Restaurant,Cocktail Bar
46,Marseille,43.3008,5.3822,80,0.0473,13,868277,2,Bay,French Restaurant,Park,Provençal Restaurant,Hotel,Historic Site,Plaza,Church,Nightclub,Multiplex
52,Mulhouse,47.7413,7.3457,75,0.0392,68,108942,2,French Restaurant,Hotel,Supermarket,Train Station,Japanese Restaurant,Rest Area,Grocery Store,Bar,Café,Restaurant
